## 1 · Imports  
Before you touch any data, import every tool you need.  
Alongside the standard scientific-Python stack, bring in **XGBoost** for gradient-boosted decision trees and **Ray** for distributed data loading and training. Ray Train’s helper classes (RunConfig, ScalingConfig, CheckpointConfig, FailureConfig) give you fault-tolerant, CPU training with almost no extra code.


In [ ]:
# 01. Imports
import os, shutil, json, uuid, tempfile, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_covtype
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb

import ray
import ray.data as rd
from ray.train import RunConfig, ScalingConfig, CheckpointConfig, FailureConfig, get_dataset_shard, get_checkpoint, get_context
from ray.train.xgboost import XGBoostTrainer, RayTrainReportCallback

### 2 · Load the University of California, Irvine (UCI) Cover type dataset  
The Cover type dataset contains ~580 000 forest-cover observations with 54 tabular features and a 7-class label. Fetch it from `sklearn.datasets`, rename the target column to `label` (Ray’s default), and shift the classes from **1-7** to **0-6** so they're zero-indexed as XGBoost expects. A quick `value_counts` sanity-check confirms the mapping worked.


In [ ]:
# 02. Load the UCI Cover type dataset (~580k rows, 54 features)
data = fetch_covtype(as_frame=True)
df = data.frame
df.rename(columns={"Cover_Type": "label"}, inplace=True)   # Ray expects "label"
df["label"] = df["label"] - 1          # 1-7  →  0-6
assert df["label"].between(0, 6).all()
print(df.shape, df.label.value_counts(normalize=True).head())

### 3 · Visualise class balance  
Highly imbalanced targets can bias tree-based models, so plot the raw label counts. The cover type distribution shows skew, but not much—the bar chart lets you judge whether extra re-scaling or class-weighting is necessary (You rely on XGBoost’s built-in handling for now).

In [ ]:
# 03. Visualize class distribution
df.label.value_counts().plot(kind="bar", figsize=(6,3), title="Cover Type distribution")
plt.ylabel("Frequency"); plt.show()

### 4 · Persist the dataset to Parquet  
Storing the data-frame once on the cluster’s shared file-system keeps later steps fast and reproducible. Parquet is columnar, compressed, and lazily readable by Ray Data, which means you can stream partitions to workers without loading everything into RAM.


In [ ]:
# 04. Write to /mnt/cluster_storage/covtype/
PARQUET_DIR = "/mnt/cluster_storage/covtype/parquet"
os.makedirs(PARQUET_DIR, exist_ok=True)
file_path = os.path.join(PARQUET_DIR, "covtype.parquet")
df.to_parquet(file_path)
print(f"Wrote Parquet -> {file_path}")

### 5 · Read the data as a Ray Dataset  
`ray.data.read_parquet` gives you a **lazy, columnar dataset** and shuffles it on-the-fly. From this point on, every split, batch, or transformation executes in parallel across the cluster, so you avoid a single-node bottleneck.

In [ ]:
# 05. Load dataset into a Ray Dataset (lazy, columnar)
ds_full = rd.read_parquet(file_path).random_shuffle()      
print(ds_full)

### 6 · Create train / validation splits  
Perform an 80 / 20 split directly on the Ray Dataset, preserving the lazy execution plan. Each subset remains a Ray Dataset object, so they can later stream to the training workers in parallel.

In [ ]:
# 06. Split to train / validation Ray Datasets
train_ds, val_ds = ds_full.split_proportionately([0.8])
print(f"Train rows: {train_ds.count()},  Val rows: {val_ds.count()}")

### 7 · Inspect a mini-batch  
Taking a tiny pandas batch helps verify that feature columns and labels have the expected shapes and types. You also build `feature_columns`, a list you reuse when building XGBoost’s `DMatrix`.


In [ ]:
# 07. Look into one batch to confirm feature dimensionality
batch = train_ds.take_batch(batch_size=5, batch_format="pandas")
print(batch.head())
feature_columns = [c for c in batch.columns if c != "label"]